In [1]:
import os
from os import path

CWD = os.getcwd()
ASSET_DIR = path.join(CWD, "../asset")
RESULT_DIR = path.join(CWD, "../.build")

WIDTH = 512
HEIGHT = 512

COLOR_MODE = "RGBA"

### Define Data Model

In [2]:
import json
import itertools
import numpy as np
from typing import List
from serde import serialize, deserialize
from dataclasses import dataclass, asdict, field
from serde.json import from_json, to_json
from serde.yaml import from_yaml
from PIL import Image, ImageDraw, ImageFont


@dataclass
class Base:
    def serialize(self):
        return asdict(self)

MODE = {
  "sourceOver": "source-over",
  "sourceIn": "source-in",
  "sourceOut": "source-out",
  "sourceAtop": "source-out",
  "destinationOver": "destination-over",
  "destinationIn": "destination-in",
  "destinationOut": "destination-out",
  "destinationAtop": "destination-atop",
  "lighter": "lighter",
  "copy": "copy",
  "xor": "xor",
  "multiply": "multiply",
  "screen": "screen",
  "overlay": "overlay",
  "darken": "darken",
  "lighten": "lighten",
  "colorDodge": "color-dodge",
  "colorBurn": "color-burn",
  "hardLight": "hard-light",
  "softLight": "soft-light",
  "difference": "difference",
  "exclusion": "exclusion",
  "hue": "hue",
  "saturation": "saturation",
  "color": "color",
  "luminosity": "luminosity",
};


@serialize
@deserialize
@dataclass
class LayerElement(Base):
    idx: int
    name: str
    file_name: str
    file_path: str
    rarity_weight: float
        
    def to_string(self) -> str:
        return f"{self.idx}{self.name}{self.file_name}{self.rarity_weight}"

@serialize
@deserialize
@dataclass
class Layer(Base):
    idx: int
    name: str
    elements: List[LayerElement]
    opacity: float = field(default=0.7)
    blend: str = field(default=MODE["sourceOver"])
        
@serialize
@deserialize
@dataclass
class Configurations(Base):
    project: str
    number_of_variations: int
    layer_order: List[Layer]
        
        
    def get_layer_order(self):
        return map(lambda x: x.name, self.layer_order)
    
    def get_combination(self):
        
        matrix = []
        for i in range(len(self.layer_order)):
            elements = []
            for j in range(len(self.layer_order[i].elements)):
                elements.append([i, j])
            matrix.append(elements)
        
        return itertools.product(*matrix)
    
tmp = json.dumps(
    {
        "project": "demo",
        "number_of_variations": 10,
        "layer_order": [
            {
                "name": "Background",
            },
            {
                "name": "Background",
                "opacity": 1,
                "blend": MODE["overlay"],
                'elements': [{'file_name': 'High#30.png',
                                'file_path': '/Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/High#30.png',
                                'idx': 0,
                                'name': 'High#30',
                                'rarity_weight': 30.0}]
            }
        ]
    }
)

### Load configurations

In [3]:
import yaml
import hashlib
from pprint import pprint


def load_layer(abs_path, data):
    
    def get_rarity_weight(x) -> float:
        _tmp = x.split("#")
        if len(_tmp) == 2:
            try:
                return float(_tmp[1])
            except ValueError:
                print("Rarity weight not macth float format")
                pass
        return 0
    
    dir_name = path.join(abs_path, data["name"])
    print(f"Load elements at dir {dir_name}")
    
    files = os.listdir(dir_name)
    
    elements = []
    for i in range(len(files)):
        
        name, ext = os.path.splitext(files[i])
                
        # Bypass file without png extension
        if ext not in ['.png']:
            continue
            
        elements.append(
            {
                "idx": i,
                "name": name,
                "file_name": files[i],
                "file_path": path.join(dir_name, files[i]),
                "rarity_weight": get_rarity_weight(name)
                
            }
        )
    return {**data, "elements": elements}
        
    
def load_configurations(
    project: str, 
    config_file: str = "config.yml"
) -> Configurations:
    """
    Function handle load yaml file then serialize to 
    Configurations Objects
    """
    dir_path = path.join(ASSET_DIR, project)
    config_file_path = path.join(dir_path, config_file)
    
#     Load master configuration
    with open(config_file_path) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
        data["project"] = project
    
    for i in range(len(data['layer_order'])):
        data['layer_order'][i]['idx'] = i
        data['layer_order'][i] = load_layer(dir_path,  data['layer_order'][i])        
    
    return from_json(Configurations, json.dumps(data))


def hash_dna(text: str) -> str:
    text = text.encode()
    hash_value = hashlib.sha1(text)
    return hash_value.hexdigest()

config = load_configurations('demo')
collection = list(config.get_combination())

Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Background
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Eyeball
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Eye color
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Iris
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Shine
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Bottom lid
Load elements at dir /Users/lap-00893/Documents/max/nft-generator-engine/notebooks/../asset/demo/Top lid


In [42]:
for data in collection:
    com = None
    features = []
    base_image = Image.new(COLOR_MODE, (WIDTH, HEIGHT), (128, 128, 128))
    for layer in data:
        layer_idx, element_idx = layer 
        layer = config.layer_order[layer_idx]
        element = layer.elements[element_idx]

        img = Image.open(element.file_path).convert(COLOR_MODE)
        if com is None:
            com = Image.alpha_composite(base_image, img)
        else:
            com = Image.alpha_composite(com, img)

        features.append(element.to_string())
    rgb_im = com.convert('RGB')

    dna = hash_dna("".join(features))
    rgb_im.save(path.join(RESULT_DIR, f'{dna}.png'), quality=95)

In [9]:
print(len(collection))
print(collection)

324
[([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 1]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 2]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 1], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 1], [6, 1]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 1], [6, 2]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 2], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 2], [6, 1]), ([0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 2], [6, 2]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 0], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 0], [6, 1]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 0], [6, 2]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 1], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 1], [6, 1]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 1], [6, 2]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 2], [6, 0]), ([0, 0], [1, 0], [2, 0], [3, 1], [4, 0], [5, 2], [6, 1]), ([0, 0], 

In [12]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import sys
from  PIL  import Image

img = cv.imread('/Users/lap-00893/Documents/max/nft-generator-engine/webcam-toy-photo2.png', cv.IMREAD_UNCHANGED)
original = img.copy()

l = int(max(5, 6))
u = int(min(6, 6))

ed = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
edges = cv.GaussianBlur(img, (21, 51), 3)
edges = cv.cvtColor(edges, cv.COLOR_BGR2GRAY)
edges = cv.Canny(edges, l, u)

_, thresh = cv.threshold(edges, 0, 255, cv.THRESH_BINARY  + cv.THRESH_OTSU)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=4)

data = mask.tolist()
sys.setrecursionlimit(10**8)
for i in  range(len(data)):
    for j in  range(len(data[i])):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
    for j in  range(len(data[i])-1, -1, -1):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
image = np.array(data)
image[image !=  -1] =  255
image[image ==  -1] =  0

mask = np.array(image, np.uint8)

result = cv.bitwise_and(original, original, mask=mask)
result[mask ==  0] =  255
cv.imwrite('bg.png', result)

img = Image.open('bg.png')
img.convert("RGBA")
datas = img.getdata()

newData = []
for item in datas:
    if item[0] ==  255  and item[1] ==  255  and item[2] ==  255:
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)

img.putdata(newData)
img.save(path.join(RESULT_DIR, "img.png"), "PNG")

In [19]:
background = Image.open('/Users/lap-00893/Documents/max/nft-generator-engine/asset/background.png').convert(COLOR_MODE)
shape = Image.open('/Users/lap-00893/Documents/max/nft-generator-engine/asset/sm_5b310c4101401.jpeg').convert(COLOR_MODE)


# com = Image.alpha_composite(background, shape)

shape_resized = shape.resize((100,100))

background.paste(shape_resized,mask=shape_resized)
background.save(path.join(RESULT_DIR, f'test.png'), quality=95)

# com = Image.composite(background, shape_resized, mask=shape_resized)
# com.save(path.join(RESULT_DIR, f'test.png'), quality=95)